In [1]:
# Song lyrics can fall under a wide variety of categories, depending on their themes, style, and the emotions they convey. Here are some common categories that song lyrics often fall into:

# ### 1. **Love and Relationships**
#    - **Romantic love**: Lyrics about falling in love, romantic experiences, or the pain of unrequited love.
#    - **Heartbreak and loss**: Songs about breakups, lost love, and emotional pain.
#    - **Longing and desire**: Expressing yearning, longing for someone, or romantic fantasies.
#    - **Friendship**: Lyrics about platonic connections, support, and loyalty.

# ### 2. **Self-Reflection and Personal Struggles**
#    - **Self-empowerment**: Lyrics about overcoming challenges, building confidence, and finding strength.
#    - **Mental health**: Songs that address depression, anxiety, and inner battles.
#    - **Identity and self-discovery**: Lyrics about finding oneself, exploring identity, or self-expression.
#    - **Regret and mistakes**: Acknowledging past errors, reflecting on personal growth, or seeking redemption.

# ### 3. **Social and Political Themes**
#    - **Revolution and protest**: Songs addressing social justice, activism, and resistance to oppression.
#    - **Inequality and injustice**: Lyrics that highlight issues such as racism, discrimination, or economic disparity.
#    - **Environment and nature**: Songs that focus on environmental concerns or the beauty of nature.
#    - **War and peace**: Themes related to conflict, loss, and the longing for peace.

# ### 4. **Celebration and Fun**
#    - **Party and enjoyment**: Lyrics celebrating fun, partying, and carefree moments.
#    - **Victory and triumph**: Songs that celebrate success, achievement, or overcoming obstacles.
#    - **Gratitude and appreciation**: Expressing thanks or joy for good times, life, or others.

# ### 5. **Philosophical and Existential**
#    - **Life and death**: Themes of mortality, the meaning of life, or the afterlife.
#    - **Fate and destiny**: Lyrics about the idea of destiny, chance, or the forces that shape our lives.
#    - **Time and change**: Reflecting on how things evolve over time or the passage of life.

# ### 6. **Storytelling and Narrative**
#    - **Ballads and folk tales**: Telling a story or recounting a narrative, often about adventure, heroism, or tragedy.
#    - **Personal stories**: Narratives drawn from real-life experiences or fictional events.
#    - **Mythology and fantasy**: Lyrics that invoke mythical creatures, legends, or fantastical worlds.

# ### 7. **Escape and Fantasy**
#    - **Dreams and imagination**: Lyrics about escapism, daydreams, or creating an ideal world.
#    - **Fantasy worlds**: Escaping to a different realm, space, or time through lyrics.
#    - **Surreal experiences**: Lyrics that take on an abstract or nonsensical tone.

# ### 8. **Spiritual and Religious**
#    - **Faith and belief**: Lyrics that explore personal spirituality, faith in a higher power, or religious teachings.
#    - **Redemption and salvation**: Themes around forgiveness, seeking redemption, or spiritual enlightenment.
#    - **Hope and grace**: Lyrics that convey a sense of divine presence, hope, or divine intervention.

# ### 9. **Cultural and Lifestyle**
#    - **Travel and exploration**: Songs about discovering new places, wandering, or seeking adventure.
#    - **Lifestyle and attitude**: Lyrics about particular subcultures, attitudes, or ways of life.
#    - **Urban life**: Themes related to city living, hustle, and the challenges of modern life.

# ### 10. **Fun and Humor**
#    - **Humor and satire**: Lyrics that are funny, ironic, or satirical, often poking fun at society or human nature.
#    - **Playfulness and quirkiness**: Lyrics that don’t take themselves too seriously and offer a light-hearted perspective.

# These categories can often overlap, with many songs incorporating themes from multiple categories at once!

In [2]:
import sys

sys.path.append('c:/users/shubh/programming_work/virtual_envs/genai_exp_env/lib/site-packages')


In [3]:
import json
import copy
import numpy as np
import os
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
import os
import time
import pandas as pd
from fastembed import TextEmbedding
import scipy.spatial.distance as spdist
from langchain_core.retrievers import BaseRetriever
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.runnables import RunnableLambda
from typing import Dict
from openai import OpenAI
import string
from collections import Counter
import re

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:

def strip_special_chars(s):
    # Using regex to remove leading and trailing non-alphanumeric characters
    return re.sub(r'^[^A-Za-z0-9]+|[^A-Za-z0-9]+$', '', s)


In [5]:
with open('openai_key.txt', 'r') as f:
    OPENAI_KEY = f.read().strip()


In [6]:
client = OpenAI(api_key = OPENAI_KEY)

In [7]:
def chat_completion(prompt, temperature = 0.1):

    global client
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {
                "role": "user",
                "content": prompt
            }
        ],
        temperature=temperature
    ).choices[0].message.content

In [8]:
def chat_completion_2(message_history, temperature = 0.1):

    global client
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=message_history,
        temperature=temperature
    ).choices[0].message.content

In [9]:
# youtube_comment_categories = [
#     "Humor and Memes",
#     "Appreciation and Praise",
#     "Questions and Curiosity",
#     "Personal Stories and Experiences",
#     "Nostalgia and Memories"

# ]


youtube_comment_categories = [
"Love and Relationships",
"Self-Reflection and Personal Struggles",
"Social and Political Themes",
"Celebration and Fun",
"Philosophical and Existential",
"Storytelling and Narrative",
"Escape and Fantasy",
"Spiritual and Religious",    
"Cultural and Lifestyle",
"Fun and Humor"
]




comment_category_2_bool = dict()
for cat in youtube_comment_categories:
    comment_category_2_bool[cat] = 0

comment_category_2_bool

{'Love and Relationships': 0,
 'Self-Reflection and Personal Struggles': 0,
 'Social and Political Themes': 0,
 'Celebration and Fun': 0,
 'Philosophical and Existential': 0,
 'Storytelling and Narrative': 0,
 'Escape and Fantasy': 0,
 'Spiritual and Religious': 0,
 'Cultural and Lifestyle': 0,
 'Fun and Humor': 0}

In [10]:
df = pd.read_csv("spotify_youtube_comments_combined.csv")

In [11]:
len(df)

80467

In [12]:
# df['unique_id'] = [_ for _ in range( len(df) )]

In [13]:
df.head()

,Unnamed: 0,Artist,Url_spotify,Track,Album,Album_type,Uri,Danceability,Energy,Key,...,Duration_ms,Url_youtube,Title,spotify_id,id,lyrics,youtube_video_id,video_id,comment,likes
0,0,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,...,222640.0,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),0d28khcov6AiegSCpG5TuT,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel...",HyHNuVaZJ-k,HyHNuVaZJ-k,I love how Gorillaz has depressing meanings to...,14760.0
1,1,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,...,222640.0,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),0d28khcov6AiegSCpG5TuT,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel...",HyHNuVaZJ-k,HyHNuVaZJ-k,It’s really crazy to realize how popular this ...,14226.0
2,2,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,...,222640.0,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),0d28khcov6AiegSCpG5TuT,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel...",HyHNuVaZJ-k,HyHNuVaZJ-k,Feel good inc. sounds like it should either be...,12339.0
3,3,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,...,222640.0,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),0d28khcov6AiegSCpG5TuT,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel...",HyHNuVaZJ-k,HyHNuVaZJ-k,The art style is just so intriguing. It’s like...,10088.0
4,4,Gorillaz,https://open.spotify.com/artist/3AA28KZvwAUcZu...,Feel Good Inc.,Demon Days,album,spotify:track:0d28khcov6AiegSCpG5TuT,0.818,0.705,6.0,...,222640.0,https://www.youtube.com/watch?v=HyHNuVaZJ-k,Gorillaz - Feel Good Inc. (Official Video),0d28khcov6AiegSCpG5TuT,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel...",HyHNuVaZJ-k,HyHNuVaZJ-k,The way Murdocs bass playing is animated is so...,9490.0


In [14]:

# comments

In [15]:
df.columns

Index(['Unnamed: 0', 'Artist', 'Url_spotify', 'Track', 'Album', 'Album_type',
       'Uri', 'Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness',
       'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo',
       'Duration_ms', 'Url_youtube', 'Title', 'spotify_id', 'id', 'lyrics',
       'youtube_video_id', 'video_id', 'comment', 'likes'],
      dtype='object')

In [16]:
df = df.drop(columns=['comment', 'Unnamed: 0'])
len(df)

80467

In [17]:
df_spotify_id_lyrics = df[ [ 'spotify_id', 'lyrics' ]  ]

In [18]:
df_spotify_id_lyrics.head()

,spotify_id,lyrics
0,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel..."
1,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel..."
2,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel..."
3,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel..."
4,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel..."


In [19]:
df = df_spotify_id_lyrics.drop_duplicates()
len(df)

12960

In [20]:
df['unique_id'] = [_ for _ in range( len(df) )]

C:\Users\shubh\AppData\Local\Temp\ipykernel_15388\4183264763.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['unique_id'] = [_ for _ in range( len(df) )]


In [21]:
df.head()

,spotify_id,lyrics,unique_id
0,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel...",0
10,1foMv2HQwfQ2vntFf9HFeG,I'm a scary gargoyle on a tower\n That you mad...,1
20,64dLd6rVqDLtkXFYrEUHIU,But in the magic cove\n There's a pretty one\n...,2
30,0q6LuUqGLUiCPP1cbdwFs3,Up on Melancholy Hill\n There's a plastic tree...,3
40,7yMiX7n9SBvadzox8T5jzT,"I ain't happy, I'm feeling glad\n I got sunshi...",4


In [22]:
len(df)

12960

In [23]:
lyrics = list(df['lyrics'])


In [24]:
df.head()

,spotify_id,lyrics,unique_id
0,0d28khcov6AiegSCpG5TuT,"Feel good\n Sha, sha-ba-da, sha-ba-da-ca, feel...",0
10,1foMv2HQwfQ2vntFf9HFeG,I'm a scary gargoyle on a tower\n That you mad...,1
20,64dLd6rVqDLtkXFYrEUHIU,But in the magic cove\n There's a pretty one\n...,2
30,0q6LuUqGLUiCPP1cbdwFs3,Up on Melancholy Hill\n There's a plastic tree...,3
40,7yMiX7n9SBvadzox8T5jzT,"I ain't happy, I'm feeling glad\n I got sunshi...",4


In [25]:
# df.to_csv("C:/Users/shubh/programming_work/git_repos/NU/courses_and_academics/data_mining/proj/refined/df_indexed_lyric.csv")

In [26]:
def check_if_categories_are_proper(llm_suggested_categories, message_history, counter, total_trials = 0):

    if total_trials==3:
        return (llm_suggested_categories, False)
    
    global youtube_comment_categories

    global required_retries_list_of_tuples 
    
    categories_list = [ strip_special_chars(category.strip(" ").strip("'").strip("\n") ) for category in llm_suggested_categories]

    list_of_remaining_categories = list(set(categories_list) - set(youtube_comment_categories))
    
    if (len(list_of_remaining_categories )>0):

        # llm_suggested_categories = chat_completion_2(prompt, temperature = 0.1).split(",")
        print(f"Requires retries - {llm_suggested_categories}")
        print(f"Requires retries list - {categories_list}")
        print(f"Requires retries remaining categories - {list_of_remaining_categories}")
        message_history.append({"role":"assistant", "content":f"{llm_suggested_categories}"})
        message_history.append({"role":"user", "content": f"Restrict the comma-separated response only to the list of relevant categories. Try again."})
        llm_suggested_categories = chat_completion_2(message_history, temperature = 0.1).strip("[").strip("]").split(",")
        llm_suggested_categories, sanity_check_passed_or_not = check_if_categories_are_proper( llm_suggested_categories, message_history, counter, total_trials = total_trials+1)
        
        required_retries_list_of_tuples.append( ( message_history, total_trials, counter ) )        
        
    else:
        return (llm_suggested_categories, True)
        
    return (llm_suggested_categories, sanity_check_passed_or_not)

In [27]:
def retry_until_you_get_correct_categories(total_trials = 0):
    pass

In [28]:
# df.to_csv("C:/Users/shubh/programming_work/git_repos/NU/courses_and_academics/data_mining/proj/refined/df_indexed.csv")

In [29]:
list_of_tuples_of_skipped_indices_and_comments = list()

In [30]:
list_of_category_2_bools_per_comment = list()
required_retries_list_of_tuples = list()

In [31]:
df.columns

Index(['spotify_id', 'lyrics', 'unique_id'], dtype='object')

In [32]:

file_path = "C:/Users/shubh/programming_work/git_repos/NU/courses_and_academics/data_mining/proj/refined/index_lyric_categories_recent.csv"


In [33]:
df_index_lyric_categories = pd.DataFrame()
does_df_exist = False

In [34]:
if ( os.path.exists(file_path) and file_path.endswith('.csv') ):
    print("found it")
    df_index_comment_categories = pd.read_csv(file_path, index_col=False)
    does_df_exist = True

else:
    print("it's not there yet")
    df_index_comment_categories = pd.DataFrame()

found it


In [35]:
counter = len(df_index_comment_categories)
# counter

In [36]:
counter

1150

In [37]:
def stack_dataframes(df1, df2):
    if df1.empty:
        return df2.reset_index(drop=True)  # Ensure a clean index reset
    return pd.concat([df1, df2], ignore_index=True)

In [38]:
def tuples_to_json(tuples_list, keys = ["counter", "llm_suggested_categories", "comment"], json_file_path = "C:/Users/shubh/programming_work/git_repos/NU/courses_and_academics/data_mining/proj/comment_classification_data/list_of_tuples_of_skipped_indices_and_lyrics_0.json"):
    dict_list = [dict(zip(keys, tup)) for tup in tuples_list]
    with open(json_file_path, 'w') as f:
        json.dump(dict_list, f, indent=4)




In [39]:
keys_for_tuples_of_skipped_indices_and_comments = ["counter", "llm_suggested_categories", "lyric"]

In [40]:
counter

1150

In [41]:
import time


In [42]:
df_recent_50 = pd.DataFrame()

for lyric in lyrics[counter:4500]:

    prompt = f"{lyric}:\n Classify this lyric into all the relevant categories from this list: - \n{youtube_comment_categories}\nReturn the response as a list of comma-separated categories."

    # print(prompt)
    
    comment_category_2_bool = dict()
    comment_category_2_bool['unique_id'] = counter
    comment_category_2_bool['lyric'] = lyric
    
    for cat in youtube_comment_categories:
        comment_category_2_bool[cat] = 0

    message_history = [{"role": "system", "content": "You are a helpful assistant."}]
    message_history.append( { "role": "assistant", "content": prompt } )
    
    llm_suggested_categories = chat_completion_2(message_history, temperature = 0.1).strip("[").strip("]").split(",")
    # llm_suggested_categories = strip_special_chars(llm_suggested_categories)
    # print(llm_suggested_categories)
    # print("\n")
    
    llm_suggested_categories, sanity_check_passed_or_not = check_if_categories_are_proper(llm_suggested_categories, message_history, counter, total_trials = 0)

    if not sanity_check_passed_or_not:
        
        list_of_tuples_of_skipped_indices_and_comments.append( (counter, llm_suggested_categories, lyric) )

        tuples_to_json(list_of_tuples_of_skipped_indices_and_comments)

        for category in youtube_comment_categories:
            comment_category_2_bool[category] = 0
    
        list_of_category_2_bools_per_comment.append( comment_category_2_bool )
        counter += 1
        continue
    
    categories_list = [ strip_special_chars(category.strip(" ").strip("'").strip("\n"))  for category in llm_suggested_categories]
    
    for category in categories_list:
        comment_category_2_bool[category] = 1

    list_of_category_2_bools_per_comment.append( comment_category_2_bool )
    counter += 1

    if (not counter%50):
        
        print(f"current length of list_of_category_2_bools_per_comment : {len(list_of_category_2_bools_per_comment)}")
        df_recent_50 = pd.DataFrame( list_of_category_2_bools_per_comment[-50:] )
        
        df_index_comment_categories = stack_dataframes(df_index_comment_categories, df_recent_50)
        
        # df_index_comment_categories = df_index_comment_categories.drop_duplicates()

        df_index_comment_categories.to_csv( file_path, index=False )
        print(f"time out 3 seconds")
        time.sleep(3)


Requires retries - ['Storytelling and Narrative', ' War and Conflict', ' Self-Reflection and Personal Struggles']
Requires retries list - ['Storytelling and Narrative', 'War and Conflict', 'Self-Reflection and Personal Struggles']
Requires retries remaining categories - ['War and Conflict']
Requires retries - ["'Storytelling and Narrative'", " 'War and Conflict'", " 'Self-Reflection and Personal Struggles'"]
Requires retries list - ['Storytelling and Narrative', 'War and Conflict', 'Self-Reflection and Personal Struggles']
Requires retries remaining categories - ['War and Conflict']
Requires retries - ["'Storytelling and Narrative'", " 'War and Conflict'", " 'Self-Reflection and Personal Struggles'"]
Requires retries list - ['Storytelling and Narrative', 'War and Conflict', 'Self-Reflection and Personal Struggles']
Requires retries remaining categories - ['War and Conflict']
Requires retries - ['Self-Reflection and Personal Struggles', ' Fear and Anxiety', ' Storytelling and Narrative'

In [26]:
df_comments_and_labels = pd.DataFrame(list_of_category_2_bools_per_comment)


In [28]:
df_comments_and_labels.columns

Index(['unique_id', 'Humor and Memes', 'Criticism and Complaints',
       'Appreciation and Praise', 'Questions and Curiosity',
       'Personal Stories and Experiences', 'Recommendations and Suggestions',
       'Nostalgia and Memories', 'Off-Topic or Random Comments',
       'Arguments and Debates',
       ...
       'Luis Barragan', '0:55 Escuela de Ballet Folklórico de México',
       'Agustín Hernández', '1:25 Casa Gilardi', 'Luis Barragán',
       '1:38 Cablebus Cuautepec', '2:18 Casa Estudio',
       '2:39 Nido de Quetzalcóatl', 'Javier Senosiain',
       ''Advertisements and Self-Promotion''],
      dtype='object', length=115)

In [24]:
df_comments_and_labels.to_csv("C:/Users/shubh/programming_work/git_repos/NU/courses_and_academics/data_mining/proj/df_comments_and_labels.csv")

In [25]:
list_of_category_2_bools_per_comment[-10:]

[{'unique_id': 4017,
  'Humor and Memes': 1,
  'Criticism and Complaints': 0,
  'Appreciation and Praise': 1,
  'Questions and Curiosity': 0,
  'Personal Stories and Experiences': 0,
  'Recommendations and Suggestions': 0,
  'Nostalgia and Memories': 0,
  'Off-Topic or Random Comments': 0,
  'Arguments and Debates': 0,
  'Technical Issues and Troubleshooting': 0,
  'Time Stamps and Highlights': 0,
  'Advertisements and Self-Promotion': 0,
  'Emotional Reactions': 0,
  'Song Lyrics and Quotes': 0,
  'Engagement Boosting': 0},
 {'unique_id': 4018,
  'Humor and Memes': 0,
  'Criticism and Complaints': 0,
  'Appreciation and Praise': 0,
  'Questions and Curiosity': 0,
  'Personal Stories and Experiences': 0,
  'Recommendations and Suggestions': 0,
  'Nostalgia and Memories': 0,
  'Off-Topic or Random Comments': 0,
  'Arguments and Debates': 0,
  'Technical Issues and Troubleshooting': 0,
  'Time Stamps and Highlights': 0,
  'Advertisements and Self-Promotion': 0,
  'Emotional Reactions': 0